In [1]:
pacman::p_load(readr, purrr, rlist, stringr, data.table, dplyr)

In [2]:
input_str <- "....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#..."

In [3]:
input_str <- read_file("puzzle_input.txt")

In [4]:
# nolint start
ncols <- input_str %>% str_split("\n") %>% .[[1]] %>% nchar %>% unique

input_mat <- matrix(input_str %>% str_replace_all("\n", "") %>% str_split("") %>% .[[1]], ncol = ncols, byrow = TRUE)

In [5]:
start_pos <- which(input_mat == '^', arr.ind=TRUE)

In [6]:
hashtags_pos <- which(input_mat == '#', arr.ind=TRUE) %>% as.data.table

# Part 1

In [7]:
# nolint start
find_next_location <- function(pos, hashtags_pos, direction, adjustment = 1) {
  row_current <- pos[1]
  col_current <- pos[2]
  visited[row_current, col_current] <<- TRUE
  
  if (direction == 'up') {
    next_pos <- as.matrix(hashtags_pos[col == col_current & row < row_current][order(-row)][1])
    next_pos[1] <- next_pos[1] + adjustment
    if(is.na(next_pos[1])){
      visited[1 : row_current, col_current] <<- TRUE
    } else{
      visited[next_pos[1] : row_current, col_current] <<- TRUE
    }
  } else if (direction == 'down') {
    next_pos <- as.matrix(hashtags_pos[col == col_current & row > row_current][order(row)][1])
    next_pos[1] <- next_pos[1] - adjustment
    if(is.na(next_pos[1])){
      visited[row_current:nrow(visited), col_current] <<- TRUE
    } else{
      visited[row_current:next_pos[1], col_current] <<- TRUE
    }
  } else if (direction == 'left') {
    next_pos <- as.matrix(hashtags_pos[col < col_current & row == row_current][order(-col)][1])
    next_pos[2] <- next_pos[2] + adjustment
    if(is.na(next_pos[1])){
      visited[row_current, 1:col_current] <<- TRUE
    } else{
      visited[row_current, next_pos[2]:col_current] <<- TRUE
    }
  } else if (direction == 'right') {
    next_pos <- as.matrix(hashtags_pos[col > col_current & row == row_current][order(col)][1])
    next_pos[2] <- next_pos[2] - adjustment
    if(is.na(next_pos[1])){
      visited[row_current, col_current:ncol(visited)] <<- TRUE
    } else{
      visited[row_current, col_current:next_pos[2]] <<- TRUE
    }
  } else{
    next_pos <- data.table(row = c(1), col = c(1)) %>% .[row > 1] %>% as.matrix
  }

  return(next_pos)  
}

In [8]:
specify_next_direction <- function(current_direction){
    switch(current_direction,
           'up' = 'right',
           'left' = 'up',
           'down' = 'left',
           'right' = 'down')
}

In [9]:
calculate_distance_traversed <- function(start_pos, next_pos, direction){
    row_start <- start_pos[1]
    col_start <- start_pos[2]
    
    row_next <- next_pos[1]
    col_next <- next_pos[2]
    
    if (direction %in% c('up', 'down')) {
        distance_traversed <- abs(row_next - row_start)
    } else {
        distance_traversed <- abs(col_next - col_start)
    }
    
    return(distance_traversed)
}

In [10]:
# nolint start
current_direction <- 'up'
in_map <- TRUE
current_pos <- start_pos
steps <- 0
visited <<- rep(FALSE, length(input_mat)) %>% matrix(nrow = nrow(input_mat), ncol = ncol(input_mat))

while(in_map){
    next_pos <- find_next_location(current_pos, hashtags_pos, current_direction)
    
    if(is.na(next_pos[1])){
        print("EXIT MAP")
        in_map <- FALSE
        break
    }
    distance_traversed <- calculate_distance_traversed(current_pos, next_pos, current_direction)
    current_direction <- specify_next_direction(current_direction)
    current_pos <- next_pos
    steps <- steps + distance_traversed
}

print(paste("Part 1:", sum(visited)))
print(steps)

[1] "EXIT MAP"
[1] "Part 1: 4647"
[1] 5025


# Part 2

In [11]:
hashtags_pos <- which(input_mat == '#', arr.ind=TRUE) %>% as.data.table

In [12]:
# nolint start
traverse_map <- function(start_pos, hashtags_pos){
    current_direction <- 'up'
    in_map <- TRUE
    current_pos <- start_pos
    locations_visited <<- copy(hashtags_pos)
    locations_visited[, ':='(visited_up = FALSE, visited_down = FALSE, visited_left = FALSE, visited_right = FALSE)]
    total_distance <- 0
    
    while(in_map & total_distance < 10000){
        next_pos <- find_next_location2(current_pos, hashtags_pos, current_direction, 1)
        
        if(current_direction == 'up'){
            row_adj <- -1
            col_adj <- 0
        }
        if(current_direction == 'down'){
            row_adj <- 1
            col_adj <- 0
        }
        if(current_direction == 'left'){
            row_adj <- 0
            col_adj <- -1
        }
        if(current_direction == 'right'){
            row_adj <- 0
            col_adj <- 1
        }
        
        if(is.na(next_pos[1])){
            in_map <- FALSE
            break
        }
        if(locations_visited[row == next_pos[1] + row_adj & col == next_pos[2] + col_adj, get(paste0("visited_", current_direction))]){
            break
        }
        
        locations_visited[row == next_pos[1] + row_adj & col == next_pos[2] + col_adj, paste0("visited_", current_direction)] <- TRUE
        distance_traversed <- calculate_distance_traversed(current_pos, next_pos, current_direction)
        total_distance <- total_distance + distance_traversed
        current_direction <- specify_next_direction(current_direction)
        
        current_pos <- next_pos
    }
    
    return(in_map)
}


In [13]:
path_positions <- which(visited, arr.ind = T) %>% as.data.table

In [14]:
# nolint start
# We don't need to update the visted matrix anymore since that is calculated in part 1
find_next_location2 <- function(pos, hashtags_pos, direction, adjustment = 1) {
  row_current <- pos[1]
  col_current <- pos[2]
  
  if (direction == 'up') {
    next_pos <- as.matrix(hashtags_pos[col == col_current & row < row_current][order(-row)][1])
    next_pos[1] <- next_pos[1] + adjustment

  } else if (direction == 'down') {
    next_pos <- as.matrix(hashtags_pos[col == col_current & row > row_current][order(row)][1])
    next_pos[1] <- next_pos[1] - adjustment

  } else if (direction == 'left') {
    next_pos <- as.matrix(hashtags_pos[col < col_current & row == row_current][order(-col)][1])
    next_pos[2] <- next_pos[2] + adjustment

  } else if (direction == 'right') {
    next_pos <- as.matrix(hashtags_pos[col > col_current & row == row_current][order(col)][1])
    next_pos[2] <- next_pos[2] - adjustment
  } else{
    next_pos <- data.table(row = c(1), col = c(1)) %>% .[row > 1] %>% as.matrix
  }

  return(next_pos)  
}

In [15]:
# nolint start
# results <- seq(nrow(input_mat)) %>% map_dfr(function(row){
#     seq(ncol(input_mat)) %>% map_dfr(function(col){
#         test_hastag_pos <- rbind(copy(hashtags_pos), data.table(row = row, col = col)) %>% unique
#         is_loop <- traverse_map(start_pos, test_hastag_pos)
#         result <- data.table(row = row, col = col, is_loop = is_loop)
#     })
# })

itr <<- 0
results <- path_positions[!(row == start_pos[1] - 1 & col == start_pos[2])] %>% pmap(function(...){
    dt <- data.table(...)
    test_hastag_pos <- rbind(copy(hashtags_pos), dt) %>% unique
    is_loop <- traverse_map(start_pos, test_hastag_pos)
    itr <<- itr + 1
    if(itr %% 100 == 0){
        message(itr)
    }
    is_loop
})

100

200

300

400

500

600

700

800

900

1000

1100

1200

1300

1400

1500

1600

1700

1800

1900

2000

2100

2200

2300

2400

2500

2600

2700

2800

2900

3000

3100

3200

3300

3400

3500

3600

3700

3800

3900

4000

4100

4200

4300

4400

4500

4600



In [16]:
unlist(results) %>% sum

[1] 1723